<a href="https://colab.research.google.com/github/TeamCountingStars/dataAnalysis/blob/main/%EC%9D%B4%EB%9D%BC%EC%98%A8_%EC%8A%A4%ED%83%9D%EC%98%A4%EB%B2%84%ED%94%8C%EB%A1%9C%EC%9A%B0_%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. 라이브러리 설치
!pip install google-cloud-bigquery pandas

# 2. 라이브러리 임포트 및 Colab 사용자 인증
# (이것이 .py 파일의 '서비스 계정' 방식보다 훨씬 간편한 Colab 방식)
from google.cloud import bigquery
from google.colab import auth
import pandas as pd

auth.authenticate_user()
print('Colab 사용자 인증 완료!')

project_id = 'mystack2021-2025'

# 3. BigQuery 클라이언트 초기화 (오류 확인)
try:
    client = bigquery.Client(project=project_id)
    print(f"✅ '{project_id}' 프로젝트로 BigQuery 클라이언트를 성공적으로 시작합니다.")
except Exception as e:
    print(f"❌ '{project_id}'로 클라이언트 시작 실패. ID를 확인하세요.")
    print(f"오류: {e}")

# 4. 데이터 수집 SQL 쿼리 실행
print("\n[1/1] Stack Overflow (2021-2025년) 데이터 수집 중...")

stackoverflow_sql = """
SELECT
  id,
  title,
  tags,
  creation_date
FROM
  `bigquery-public-data.stackoverflow.posts_questions`
WHERE
  creation_date BETWEEN '2021-01-01T00:00:00' AND '2025-12-31T23:59:59'
"""

# 5. 쿼리 실행 및 결과를 Pandas DataFrame으로 변환
try:
    so_raw_df = client.query(stackoverflow_sql).to_dataframe()

    print(f"\n✅ 수집 완료! 총 {len(so_raw_df)}개의 질문을 'so_raw_df'에 로드했습니다.")

    # 6. 수집된 데이터 확인
    print("\n--- 수집된 원본 데이터 샘플 ---")
    print(so_raw_df.head())

except Exception as e:
    print("\n❌ 쿼리 실행 중 오류가 발생했습니다.")
    print("--- [오류 원인 확인] ---")
    print("1. GCP 'project_id'가 올바른가요?")
    print("2. 'BigQuery API'가 [사용]으로 설정되었나요?")
    print("3. 프로젝트에 '결제 계정'이 연결되었나요?")
    print(f"\n상세 오류 메시지: {e}")



Colab 사용자 인증 완료!
✅ 'mystack2021-2025' 프로젝트로 BigQuery 클라이언트를 성공적으로 시작합니다.

[1/1] Stack Overflow (2021-2025년) 데이터 수집 중...

✅ 수집 완료! 총 2898368개의 질문을 'so_raw_df'에 로드했습니다.

--- 수집된 원본 데이터 샘플 ---
         id                                              title  \
0  73250763  Error CS0246: The type or namespace name 'Stre...   
1  73321062       How to install specific SDK (I need 6.0.300)   
2  73511481  Graphql Permessions for Ticket Only And Admins...   
3  73519558    Divi custom header on phone not matching others   
4  73282909  How can I find an old API Google Maps Address ...   

                                   tags                    creation_date  
0                            c#|unity3d 2022-08-05 13:43:25.850000+00:00  
1  .net|sdk|windows-subsystem-for-linux 2022-08-11 12:58:42.827000+00:00  
2   amazon-web-services|graphql|amplify 2022-08-27 13:42:09.123000+00:00  
3                           header|divi 2022-08-28 14:31:09.600000+00:00  
4                                   ap

In [ ]:
so_raw_df

,id,title,tags,creation_date
0,73250763,Error CS0246: The type or namespace name 'Stre...,c#|unity3d,2022-08-05 13:43:25.850000+00:00
1,73321062,How to install specific SDK (I need 6.0.300),.net|sdk|windows-subsystem-for-linux,2022-08-11 12:58:42.827000+00:00
2,73511481,Graphql Permessions for Ticket Only And Admins...,amazon-web-services|graphql|amplify,2022-08-27 13:42:09.123000+00:00
3,73519558,Divi custom header on phone not matching others,header|divi,2022-08-28 14:31:09.600000+00:00
4,73282909,How can I find an old API Google Maps Address ...,api,2022-08-08 19:11:09.300000+00:00
...,...,...,...,...
2898363,73133211,How to access iterating variables of generator...,python,2022-07-27 06:31:35.880000+00:00
2898364,72956076,Why can't I return it to the str() built-in fu...,python,2022-07-12 17:14:20.370000+00:00
2898365,73033054,Delete all sublists of nested list of list,python,2022-07-19 07:48:25.487000+00:00
2898366,72969833,Comparing against unicode and str,python,2022-07-13 16:45:45.230000+00:00


In [ ]:
# 'so_raw_df' DataFrame을 CSV 파일로 저장
so_raw_df.to_csv('stackoverflow_raw_2021_2025.csv', index=False)

print("✅ 'stackoverflow_raw_2021_2025.csv' 파일로 저장이 완료되었습니다.")


KeyboardInterrupt: 

In [ ]:
# --- 1. 중복 제거 ---

print(f"처리 전 원본 데이터: {len(so_raw_df)} 행")

# 'id' 열 기준 중복 확인
duplicate_count = so_raw_df['id'].duplicated().sum()
print(f"'id' 기준 중복된 행 개수: {duplicate_count}")

if duplicate_count > 0:
    # 'id' 열 기준으로 중복된 행 제거 (첫 번째 값 남김)
    so_raw_df = so_raw_df.drop_duplicates(subset=['id'], keep='first')
    print(f"중복 제거 후 데이터: {len(so_raw_df)} 행")
else:
    print("중복된 행이 없어 제거 작업을 건너뜁니다.")


처리 전 원본 데이터: 2898368 행
'id' 기준 중복된 행 개수: 0
중복된 행이 없어 제거 작업을 건너뜁니다.


In [ ]:
# --- 2. 결측치 처리 ---

# 1. 현재 결측치 현황 확인
print("\n--- 결측치 확인 (처리 전) ---")
print(so_raw_df.isnull().sum())

# 2. 결측치 제거 (분석에 필수적인 열 기준)
# 'id' 또는 'creation_date'가 없는 데이터는 분석 가치가 없음
# 'tags'가 없는 데이터는 'AI_field' 분류가 불가능하므로 제거
essential_columns = ['id', 'creation_date', 'tags']
original_count = len(so_raw_df)

so_raw_df = so_raw_df.dropna(subset=essential_columns)

removed_count = original_count - len(so_raw_df)
print(f"\n필수 열({essential_columns})의 결측치로 인해 제거된 행: {removed_count} 개")

# 3. 최종 결과 확인
print("\n--- 최종 데이터 정보 (처리 후) ---")
print(f"최종 데이터: {len(so_raw_df)} 행")
print(so_raw_df.info())



--- 결측치 확인 (처리 전) ---
id               0
title            0
tags             0
creation_date    0
dtype: int64

필수 열(['id', 'creation_date', 'tags'])의 결측치로 인해 제거된 행: 0 개

--- 최종 데이터 정보 (처리 후) ---
최종 데이터: 2898368 행
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2898368 entries, 0 to 2898367
Data columns (total 4 columns):
 #   Column         Dtype              
---  ------         -----              
 0   id             Int64              
 1   title          object             
 2   tags           object             
 3   creation_date  datetime64[us, UTC]
dtypes: Int64(1), datetime64[us, UTC](1), object(2)
memory usage: 91.2+ MB
None


In [ ]:
# 현재 DataFrame의 총 행 (row) 개수를 출력
print(len(so_raw_df))


2898368


In [ ]:
import numpy as np

# 1. 'creation_date'를 날짜 타입(datetime)으로 변환 (오류 방지)
#    - errors='coerce'는 변환 불가능한 날짜가 있으면 NaT(결측치)로 만듭니다.
print("날짜 타입 변환 중...")
so_raw_df['creation_date'] = pd.to_datetime(so_raw_df['creation_date'], errors='coerce')

# 2. 'tags'를 문자열 타입으로 변환하고 모두 소문자로 변경 (오류 방지)
print("태그 타입 변환 및 소문자화 중...")
so_raw_df['tags_processed'] = so_raw_df['tags'].astype(str).str.lower()

print("사전 준비 완료.")

날짜 타입 변환 중...
태그 타입 변환 및 소문자화 중...
사전 준비 완료.


In [ ]:
# 2. 파생 변수 1: `period` 생성 (✅ 수정된 코드)

# 기준이 되는 날짜 (ChatGPT 출시일)
# -------------------------------------------------------------
# [수정] pd.to_datetime에 utc=True를 추가하여
#       'creation_date'와 동일한 UTC(tz-aware) 타입으로 만듬.
# -------------------------------------------------------------
pivot_date = pd.to_datetime('2022-11-30', utc=True)

# 'creation_date'가 기준일(pivot_date)보다 작거나 같으면 'Before', 아니면 'After'
# (사전 준비 단계에서 날짜 타입 변환 시 생긴 결측치(NaT)는 'After'로 분류)
so_raw_df['period'] = np.where(so_raw_df['creation_date'] <= pivot_date,
                                      'Before',
                                      'After')

print("'period' 변수 생성 완료.")

# 생성된 값 확인
print(so_raw_df['period'].value_counts())

'period' 변수 생성 완료.
period
Before    2898368
Name: count, dtype: int64


In [ ]:
# 1. 태그(tags)에 포함된 키워드를 기반으로 조건 리스트
#    (Stack Overflow에서 실제 사용되는 태그 기준으로 구성)
conditions = [
    so_raw_df['tags_processed'].str.contains('llm|large-language-model|gpt|transformer|langchain|bert'),
    so_raw_df['tags_processed'].str.contains('reinforcement-learning|rl'),
    so_raw_df['tags_processed'].str.contains('computer-vision|opencv|yolo|image-processing'),
    so_raw_df['tags_processed'].str.contains('nlp|natural-language-processing|nltk|spacy|word-embedding'),
    so_raw_df['tags_processed'].str.contains('machine-learning|scikit-learn|tensorflow|pytorch|keras|pandas|numpy')
]

# 2. 각 조건에 해당하는 분류 이름 리스트 (순서 동일)
choices = [
    'LLM',
    'Reinforcement Learning',
    'Computer Vision',
    'NLP',
    'General ML'
]

# 3. np.select를 사용해 'AI_field' 열 생성
#    - conditions에 모두 해당하지 않으면 'Other'로 분류
so_raw_df['AI_field'] = np.select(conditions, choices, default='Other')

In [ ]:
# ChatGPT, Copilot 등 AI 코드 생성 도구 관련 태그로 대체합니다.
vibe_coding_tags = 'vibe-coding|github-copilot|generative-ai|chatgpt|openai-api'
ai_agent_tags = 'ai-agent|langchain|autogen|agent'
autonomous_tags = 'autonomous|self-driving|autonomous-vehicles'

# 각 키워드 그룹을 포함하는지 여부를 True/False로 저장
so_raw_df['has_vibe_coding_tag'] = so_raw_df['tags_processed'].str.contains(vibe_coding_tags)
so_raw_df['has_ai_agent_tag'] = so_raw_df['tags_processed'].str.contains(ai_agent_tags)
so_raw_df['has_autonomous_tag'] = so_raw_df['tags_processed'].str.contains(autonomous_tags)

In [ ]:
# 1. 상위 5개 데이터를 출력해 새 열들이 잘 생성됐는지 확인
print("\n--- 파생 변수 생성 후 상위 5개 데이터 ---")
print(so_raw_df.head())

# 2. 각 파생 변수의 값들이 몇 개씩 생성되었는지 확인 (분포 확인)
print("\n--- 'period' 열 분포 ---")
print(so_raw_df['period'].value_counts())

print("\n--- 'AI_field' 열 분포 ---")
print(so_raw_df['AI_field'].value_counts())

print("\n--- 'has_vibe_coding_tag' 열 분포 ---")
print(so_raw_df['has_vibe_coding_tag'].value_counts())


--- 파생 변수 생성 후 상위 5개 데이터 ---
         id                                              title  \
0  73250763  Error CS0246: The type or namespace name 'Stre...   
1  73321062       How to install specific SDK (I need 6.0.300)   
2  73511481  Graphql Permessions for Ticket Only And Admins...   
3  73519558    Divi custom header on phone not matching others   
4  73282909  How can I find an old API Google Maps Address ...   

                                   tags                    creation_date  \
0                            c#|unity3d 2022-08-05 13:43:25.850000+00:00   
1  .net|sdk|windows-subsystem-for-linux 2022-08-11 12:58:42.827000+00:00   
2   amazon-web-services|graphql|amplify 2022-08-27 13:42:09.123000+00:00   
3                           header|divi 2022-08-28 14:31:09.600000+00:00   
4                                   api 2022-08-08 19:11:09.300000+00:00   

                         tags_processed  period AI_field  has_vibe_coding_tag  \
0                            c#|uni

In [ ]:
# 1. 파일 이름 설정
output_filename = 'stackoverflow_all_years_cleaned.csv'

# 2. CSV 파일로 저장
#    index=False는 0, 1, 2, ... 같은 불필요한 인덱스 열이 저장되지 않게 함.
print(f"'{output_filename}' 파일로 저장 중...")

so_raw_df.to_csv(output_filename, index=False)

print("저장 완료!")

'stackoverflow_all_years_cleaned.csv' 파일로 저장 중...
저장 완료!
